<img src="imagenes/iebs-logo.jpg" alt="Logo IEBS" align="center">
<br><br>
<h1><font color="#113D68" size=5>Impacto de las partículas contaminantes en la temperatura y calidad del aire en el Valle de Aburrá,  Antioquia-Colombia.</font></h1>

<h1><font color="#113D68" size=6>Carga de la temperatura y datos de las estaciones - Paso 3</font></h1>

<br><br>
<div style="text-align: right">
<font size=3>Laura López Bedoya</font><br>
<font size=3>Andrés Fernando Morales González</font><br>
<font size=3>Proyecto final del Postgrado en Data Science y Machine Learning</font><br>
<font size=3>2019 - 2020</font>
</div>

<a href="Paso 4 - Analisis de los datos.html" target="_self">→ Ir al Paso 4 - Analisis de los datos</a> <br> <br>
<a href="Paso 2 - Carga de material particulado.html" target="_self">← Regresar al Paso 2 - Carga de material particulado</a>

La temperatura y los datos de las estaciones se encuentran en formato JSON, por lo que necesitamos convertirlos a Dataframes y archivos CSV para persistirlos mientras los cargamos a la base de datos

In [1]:
# Imports
import pandas as pd
import pymysql
import os
import json

### Conexión a la Base de datos para obtener las estaciones
Debido a que no todas las estaciones tienen la información sobre el material particulado es necesario saber cuales si, para cargar la temperatura correspondiente a cada estación

In [2]:
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

### Consulta de la lista de estaciones

In [3]:
# Se consultan los codigos de las estaciones que si tienen la informacion sobre el material particulado
connection = pymysql.connect(host, user, password, database)
with connection.cursor() as cursor:
    query = "select distinct codigoSerial from CALIDAD_AIRE order by codigoSerial asc"
    cursor.execute(query);
    listaEstaciones = cursor.fetchall()
connection.close()

### Lectura de JSON y creación del DataFrame

In [4]:
# Lectura de estaciones para obtener la longitud, latitud y altitud
with open("estaciones.json") as json_file:
    estaciones = json.load(json_file)
estaciones

[{'PM2_5_CC_ICA': 80.36793320234932,
  'PM2_5_mean': 27.923704831397917,
  'PM2_5_last': 8.31825226575,
  'fecha_hora': '2020-08-05T14:00:00',
  'humedad_relativa': 34.8211666667,
  'latitude': 6.26792,
  'longitude': -75.54586359999996,
  'temperatura': 31.6323333333,
  'online': 'Y',
  'altitud': 1775,
  'barrio': 'El Raizal',
  'vereda': 'Zona Urbana',
  'ciudad': 'Medellin',
  'estado': 'A',
  'nombre': '1',
  'codigo': 1},
 {'PM2_5_CC_ICA': 71.16879708404518,
  'PM2_5_mean': 23.249041783525,
  'PM2_5_last': 5.53828634788,
  'fecha_hora': '2020-08-05T14:00:00',
  'humedad_relativa': 41.3533333333,
  'latitude': 6.2550666,
  'longitude': -75.61927170000001,
  'temperatura': 29.077,
  'online': 'Y',
  'altitud': 1584,
  'barrio': 'San Javier No.1',
  'vereda': 'Zona Urbana',
  'ciudad': 'Medellin',
  'estado': 'A',
  'nombre': '2',
  'codigo': 2},
 {'PM2_5_CC_ICA': 67.39653494483835,
  'PM2_5_mean': 21.332116737275,
  'PM2_5_last': 10.2519612208,
  'fecha_hora': '2020-08-05T14:00:00'

In [5]:
def buscarEstacionPorCodigo(codigo):
    return next(item for item in estaciones if item["codigo"] == codigo)

In [6]:
# Metodo que lee la temperatura de un dia de una estacion
def leerTemperaturaDia(rutaArchivo, codigo):
    with open(rutaArchivo) as json_file:
        medicionDia = json.load(json_file)
        dia = medicionDia["date"].replace("/", "-")
        estacion = buscarEstacionPorCodigo(codigo)
        df = pd.DataFrame({
            "codigoSerial": [codigo] * 24,
            "temperatura": medicionDia["data"],
            "Fecha_Hora": ["{0} {1}:00:00".format(dia, hora) for hora in range(0,24)],
            "longitud": [estacion["longitude"]] * 24,
            "latitud": [estacion["latitude"]] * 24,
            "altitud": [estacion["altitud"]] * 24
        })
        return df
    
# Se guarda en un dataframe la temperatura de todas las estaciones desde Junio de 2019 a Junio de 2020
df = pd.DataFrame()
for codigo in listaEstaciones:
    ruta = "./temperaturaV2/{0}".format(codigo[0])
    try:
        listaArchivos = os.listdir(ruta)
        for archivo in listaArchivos:
            rutaArchivo = ruta + "/" + archivo
            df2 = leerTemperaturaDia(rutaArchivo, codigo[0])
            df = df.append(df2, ignore_index=True)
    except OSError as e:
        print("el archivo " + ruta + " no existe!")

el archivo ./temperaturaV2/6 no existe!


### Guardado del DataFrame en CSV

In [10]:
df.to_csv(r'temperaturas_totales.csv', index = False, encoding='utf-8')

In [12]:
df.sample(10)

,codigoSerial,temperatura,Fecha_Hora,longitud,latitud,altitud
71667,25,20.646167,2018-8-19 3:00:00,-75.447390,6.380560,1415
438254,82,-9999.000000,2017-6-12 14:00:00,-75.506908,6.349324,1444
60714,12,27.908696,2018-7-17 18:00:00,-75.604396,6.225369,1584
430718,81,-9999.000000,2017-3-31 14:00:00,-75.593093,6.273629,1600
515968,85,-9999.000000,2018-2-11 16:00:00,-75.590030,6.157710,1622
640516,94,17.690833,2018-10-9 4:00:00,-75.647020,6.154190,1821
555667,87,-9999.000000,2017-2-6 19:00:00,-75.559705,6.330849,1481
12156,3,25.178167,2018-8-3 12:00:00,-75.539352,6.330812,1525
236855,43,-9999.000000,2017-1-19 23:00:00,-75.580685,6.292195,1696
626862,92,16.701333,2019-6-1 6:00:00,-75.611440,6.190230,1730


In [13]:
# Actualización de los valores en la base de datos

queryFormat = "update CALIDAD_AIRE set temperatura = {0}, longitud = {1}, latitud = {2}, altitud = {3} where codigoSerial = {4} and Fecha_Hora = STR_TO_DATE('{5}','%Y-%m-%d %H:%i:%s');"
connection = pymysql.connect(host, user, password, database)
try:
    with connection.cursor() as cursor:
        contador = 0
        for ind in df.index:
            temperatura = df["temperatura"][ind]
            longitud = df["longitud"][ind]
            latitud = df["latitud"][ind]
            altitud = df["altitud"][ind]
            codigoSerial = df["codigoSerial"][ind]
            fecha_Hora = df["Fecha_Hora"][ind]
            query = queryFormat.format(temperatura, longitud, latitud, altitud, codigoSerial, fecha_Hora)
            cursor.execute(query)
            connection.commit()
            contador += 1
        print("Se afectaron {0} columnas, el tamaño del DataFrame es {1}".format(contador, len(df)))
except Exception as ex:
    print(ex)
finally:
    connection.close()

Se afectaron 677784 columnas, el tamaño del DataFrame es 677784


<a href="Paso 4 - Analisis de los datos.html" target="_self">→ Ir al Paso 4 - Analisis de los datos</a> <br> <br>
<a href="Paso 2 - Carga de material particulado.html" target="_self">← Regresar al Paso 2 - Carga de material particulado</a>